In [89]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.impute import KNNImputer

# Classificadores:
from sklearn.neighbors import NearestNeighbors

## 1) Loading Data

In [90]:
df_meteorologia = pd.read_csv("..\Dados\Views\dados_meteorologicos.csv", index_col=0)
df_produto_agricola = pd.read_csv(R"..\Dados\Tabela_final\dados_producao_agricola.csv", index_col=0)
df_municipios_sertao = pd.read_csv(r"..\Dados\Views\municipios_sertao.csv", index_col=0)
df_transporte = pd.read_csv(r"..\Dados\Views\custo_de_transporte.csv", index_col=0)
df_recursos_hidricos = pd.read_csv(r"..\Dados\Views\recursos_hidricos.csv", index_col=0)
df_solo = pd.read_csv(r"..\Dados\Views\solos_municipios.csv", index_col=0)

In [91]:
df_classificador = df_municipios_sertao[["IBGE7", "NOME", "LATITUDE", "LONGITUDE"]].copy()

In [92]:
df_classificador.head()

,IBGE7,NOME,LATITUDE,LONGITUDE
0,2700300,ARAPIRACA,-9.75487,-36.6615
1,2700706,BATALHA,-9.67420,-37.1330
2,2700904,BELO MONTE,-9.82272,-37.2770
3,2701209,CACIMBINHAS,-9.40121,-36.9911
4,2701605,CANAPI,-9.11932,-37.5967


In [93]:
# Merges com o dataframe do classificador:
df_classificador = df_classificador.merge(df_transporte, on="IBGE7", how="left")
df_classificador = df_classificador.merge(df_meteorologia, on="IBGE7", how="left")
df_classificador = df_classificador.merge(df_recursos_hidricos[["IBGE7", "AREA_IRRIGADA_TOT"]], on="IBGE7", how="left")

# Merge do dataframe de produtos agrícolas com o de municípios do sertão:
df_produto_agricola = df_produto_agricola.merge(df_municipios_sertao[["IBGE7", "NOME"]], how="inner", on="NOME")

In [94]:
df_solo.head()

,IBGE7,SOLO,AREA_TOTAL
0,2207959,LAd - Latossolos Amarelos Distroficos,64739.067171
1,2207959,PVAe - Argissolos Vermelho-Amarelos Eutroficos,4865.805456
2,2207959,RQo - Neossolos Quartzarenicos Orticos,24660.252674
3,2207934,PVAe - Argissolos Vermelho-Amarelos Eutroficos,23145.966454
4,2207934,RQo - Neossolos Quartzarenicos Orticos,65225.223645


In [95]:
df_produto_agricola["PRODUTO"].unique()

array(['ALGODAO HERBACEO (EM CAROCO)', 'AMENDOIM (EM CASCA)', 'BANANA ',
       'CASTANHA DE CAJU', 'FAVA (EM GRAO)', 'FEIJAO (EM GRAO)',
       'MANDIOCA', 'MANGA', 'MILHO (EM GRAO)', 'TOMATE',
       'CAFE (EM GRAO) ARABICA', 'CAFE (EM GRAO) TOTAL', 'MELANCIA',
       'MELAO', 'BATATA-DOCE', 'BATATA-INGLESA',
       'CAFE (EM GRAO) CANEPHORA', 'URUCUM ', 'CACAU (EM AMENDOA)',
       'SOJA (EM GRAO)', 'SORGO (EM GRAO)', 'MAMONA ', 'UVA',
       'PIMENTA-DO-REINO', 'TRIGO (EM GRAO)', 'GUARANA '], dtype=object)

In [96]:
# Obtendo o produto, respectivamente, de maior Valor e maior rendimento.
rendimentos_max = df_produto_agricola.groupby(["NOME"]).agg({"REND_MEDIO":"max"}).reset_index()
df_agro = rendimentos_max.merge(df_produto_agricola[["NOME", "IBGE7", "REND_MEDIO", "PRODUTO", "VALOR_PROD"]], on=["NOME", "REND_MEDIO"], how="inner")
df_agro = df_agro.drop_duplicates(["NOME", "REND_MEDIO", "PRODUTO"])
max = df_agro.groupby("NOME")["VALOR_PROD"].idxmax()
df_agro_max = df_agro.loc[max].reset_index(drop=True)
df_agro_max.drop(columns=["NOME"], inplace=True)
df_classificador = df_classificador.merge(df_agro_max, on="IBGE7", how="inner")
df_classificador.drop(columns=["REND_MEDIO", "VALOR_PROD"], inplace=True)
df_classificador.rename(columns={"PRODUTO": "PRODUTO_MAIOR_VALOR"},inplace=True)

In [97]:
df_classificador.head()

,IBGE7,NOME,LATITUDE,LONGITUDE,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,PRODUTO_MAIOR_VALOR
0,2700300,ARAPIRACA,-9.75487,-36.6615,24042.166072,988.689803,1051.169614,26.157050,1.553386,236.84,327.176525,TOMATE
1,2700706,BATALHA,-9.67420,-37.1330,24257.110578,713.023256,1318.900553,27.754333,1.427155,20.83,0.000000,MELANCIA
2,2700904,BELO MONTE,-9.82272,-37.2770,24226.863743,713.023256,1318.900553,27.754333,1.427155,20.83,0.000000,BATATA-DOCE
3,2701209,CACIMBINHAS,-9.40121,-36.9911,24358.206538,209.280265,1585.450518,26.482644,1.062182,278.01,0.000000,MILHO (EM GRAO)
4,2701605,CANAPI,-9.11932,-37.5967,24734.221802,551.553841,1437.777193,27.047011,2.518969,187.00,0.000000,TOMATE


In [98]:
# Obtendo solo de maior área dentro do município
index_solo_max = df_solo.groupby("IBGE7")["AREA_TOTAL"].idxmax()
df_solo_max = df_solo.loc[index_solo_max]
df_classificador = df_classificador.merge(df_solo_max, how="inner", on="IBGE7")
df_classificador.rename(columns={"SOLO":"SOLO_PREDO"}, inplace=True)

In [101]:
df_classificador.head()

,IBGE7,NOME,LATITUDE,LONGITUDE,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,PRODUTO_MAIOR_VALOR,SOLO_PREDO,AREA_TOTAL
0,2700300,ARAPIRACA,-9.75487,-36.6615,24042.166072,988.689803,1051.169614,26.157050,1.553386,236.84,327.176525,TOMATE,LAd - Latossolos Amarelos Distroficos,35700.513746
1,2700706,BATALHA,-9.67420,-37.1330,24257.110578,713.023256,1318.900553,27.754333,1.427155,20.83,0.000000,MELANCIA,RLe - Neossolos Litolicos Eutroficos,27053.475931
2,2700904,BELO MONTE,-9.82272,-37.2770,24226.863743,713.023256,1318.900553,27.754333,1.427155,20.83,0.000000,BATATA-DOCE,RLe - Neossolos Litolicos Eutroficos,33004.844277
3,2701209,CACIMBINHAS,-9.40121,-36.9911,24358.206538,209.280265,1585.450518,26.482644,1.062182,278.01,0.000000,MILHO (EM GRAO),SXe - Planossolos Haplicos Eutroficos,26836.700678
4,2701605,CANAPI,-9.11932,-37.5967,24734.221802,551.553841,1437.777193,27.047011,2.518969,187.00,0.000000,TOMATE,SXe - Planossolos Haplicos Eutroficos,32083.122951


## 2) Préprocessamento

In [30]:
X = df_classificador.drop(columns=["IBGE7", "NOME", "PRODUTO_MAIOR_VALOR"])
y = df_classificador["PRODUTO_MAIOR_VALOR"].values

In [613]:
# Onehot ecoding:
#one_hot = OneHotEncoder()
#y_one_hot = one_hot.fit_transform(y.reshape(-1, 1)).toarray()
#labels = one_hot.get_feature_names_out()

In [31]:
colunas_num = X.columns 

In [32]:
min_max = MinMaxScaler()
X_min_max = min_max.fit_transform(X)

In [35]:
imputer = KNNImputer(n_neighbors=5)
X_imputado = imputer.fit_transform(X_min_max)

## 3) Treinamento do KNN

In [38]:
k_neighburs = 5 
knn = NearestNeighbors(n_neighbors=k_neighburs, n_jobs=-1, metric="cosine")
knn.fit(X_imputado)

NearestNeighbors(metric='cosine', n_jobs=-1)

In [60]:
id_municipio = 2702009
distance, neighbours_indices = knn.kneighbors(X_imputado[16].reshape(1, -1), n_neighbors=k_neighburs+1)

In [76]:
distancias_corrigidas = distance[0][1:]
neighbours_indices_corrigidos = neighbours_indices[0][1:]

In [79]:
df_classificador.iloc[neighbours_indices_corrigidos]

,IBGE7,NOME,LATITUDE,LONGITUDE,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,PRODUTO_MAIOR_VALOR
11,2702900,GIRAU DO PONCIANO,-9.88404,-36.8316,24031.204758,988.689803,1051.169614,26.15705,1.553386,236.84,0.000000,TOMATE
0,2700300,ARAPIRACA,-9.75487,-36.6615,24042.166072,988.689803,1051.169614,26.15705,1.553386,236.84,327.176525,TOMATE
6,2702009,COITE DO NOIA,-9.63348,-36.5845,24082.577828,988.689803,1051.169614,26.15705,1.553386,236.84,0.000000,TOMATE
36,2709202,TRAIPU,-9.96262,-37.0071,24050.952763,988.689803,1051.169614,26.15705,1.553386,236.84,0.000000,MANDIOCA
7,2702355,CRAIBAS,-9.61780,-36.7697,24157.623963,988.689803,1051.169614,26.15705,1.553386,236.84,0.000000,TOMATE


In [81]:
df_classificador.loc[[16]]

,IBGE7,NOME,LATITUDE,LONGITUDE,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,PRODUTO_MAIOR_VALOR
16,2704104,LAGOA DA CANOA,-9.83291,-36.7413,24027.205932,988.689803,1051.169614,26.15705,1.553386,236.84,55.965723,MANDIOCA
